In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random


In [2]:
loader = DirectoryLoader(os.getcwd() + "/data_input", show_progress=True)
documents = loader.load()

100%|██████████| 1/1 [00:04<00:00,  4.09s/it]


In [3]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)

Number of chunks =  23
An extensive literature search was performed, and 56 articles published in peer-reviewed journals between 2005 and 2021 were selected and analyzed. The corresponding authors' experiential knowledge served as the foundation for the analysis.


# Create Dataframe for all Chunks

In [4]:
from df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(23, 3)


,text,source,chunk_id
0,Abstract India’s health indicators have improv...,/home/ec2-user/llm_knowledge_graph/data_input/...,c04a41953c1d404f869a05a850ae8e00
1,"Categories: Public Health, Epidemiology/Public...",/home/ec2-user/llm_knowledge_graph/data_input/...,8a99a3d88d204a46aa7899e633219a52
2,Introduction And Background India’s health ind...,/home/ec2-user/llm_knowledge_graph/data_input/...,67036f81bb5f48908f6ae4d04488579c
3,"An extensive literature search was performed, ...",/home/ec2-user/llm_knowledge_graph/data_input/...,8a96401c5417482f862d320eea4c2737
4,Review Overview of the public and private heal...,/home/ec2-user/llm_knowledge_graph/data_input/...,f428df71729245bb98ddd7f73097d2a8
